In [19]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import spacy
from sklearn.naive_bayes import MultinomialNB
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
import nltk
from nltk.corpus import sentiwordnet as swn
import os
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
import nltk
from nltk.corpus import sentiwordnet as swn
import re
from nltk.tokenize import word_tokenize
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import feature_extraction,model_selection,metrics

In [20]:
dataset = pd.read_csv('data-1_train.csv')

In [21]:
test=pd.read_csv('data-1_test.csv')

In [22]:
dataset.head()

,example_id,text,aspect_term,term_location,class
0,2333_1,Obviously one of the most important features o...,human interface,69--84,0
1,1805_1,Good for every day computing and web browsing.,every day computing,9--28,1
2,2782_2,while the keyboard itself is alright[comma] th...,mouse command buttons,115--136,-1
3,1385_0,Again[comma] the same problem[comma] the right...,right speaker,29--42,-1
4,1423_0,My problem was with DELL Customer Service.,DELL Customer Service,20--41,-1


In [23]:
def train_SVC(x, y):

    SVC=LinearSVC(multi_class='crammer_singer',random_state=0)
    preds = model_selection.cross_val_predict(SVC, x, y, cv=10)
    Score = metrics.accuracy_score(y,preds)
    labels = [-1, 0, 1]
    precision = metrics.precision_score(y,preds,average=None,labels=labels)
    recall = metrics.recall_score(y,preds,average=None,labels=labels)
    f1Score = metrics.f1_score(y,preds,average=None,labels=labels)


    print("\nAcurracy : ",Score*100," %\n")
    for i in range(len(labels)):
        print("Precision Of the %s class: %f" %(labels[i],precision[i]))
        print("Recall Of the %s class: %f" %(labels[i],recall[i]))
        print("F1-Score Of the %s class: %f" %(labels[i],f1Score[i]),"\n\n")
    SVC.fit(x,y)
    pickle.dump(SVC, open('svc_model', 'wb'))
    return SVC


In [24]:
def loadSentimentScoresBingLiuLexicon():
    posfile = open("positive-words.txt","r")
    positive = set(posfile.read().split("\n"))
    negfile = open("negative-words.txt","r")
    negative = set(negfile.read().split("\n"))
    return positive,negative

In [25]:
def getContextWords(sentence, start, end):
    left = sentence[:start]
    right = sentence[end:]
    left_tokens = word_tokenize(left)
    right_tokens = word_tokenize(right)
    reduced_tokens = left_tokens[-5:]+right_tokens[:5]
    return " ".join(reduced_tokens)

In [26]:
def getSentimentScoresfromLexicon(file_name):
    fileid = open(file_name,"r",encoding="utf8")
    review_scores = {}
    for line in fileid.read().split("\n"):
        if not line == " ":
            terms = line.split(",")
            review_scores.update({terms[0]:float(terms[1])})
    return review_scores

In [27]:
#POS Tagging and Lemmatization
def getSentimentScoresSentiwordNet(reduced_sentences):
    stokens = [nltk.word_tokenize(sent) for sent in reduced_sentences]
    taggedlist = []
    for stoken in stokens:
        taggedlist.append(nltk.pos_tag(stoken))
    wnl = nltk.WordNetLemmatizer()

    score_list=[]
    for idx,taggedsent in enumerate(taggedlist):
        score_list.append([])
        for idx2,t in enumerate(taggedsent):
            newtag=''
            lemmatized=wnl.lemmatize(t[0])
            if t[1].startswith('NN'):
                newtag='n'
            elif t[1].startswith('JJ'):
                newtag='a'
            elif t[1].startswith('V'):
                newtag='v'
            elif t[1].startswith('R'):
                newtag='r'
            else:
                newtag=''       
            if(newtag!=''):    
                synsets = list(swn.senti_synsets(lemmatized, newtag))
                #Getting average of all possible sentiments, as you requested        
                score=0
                if(len(synsets)>0):
                    for syn in synsets:
                        score+=syn.pos_score()-syn.neg_score()
                    score_list[idx].append(score/len(synsets))
                else:
                    score_list[idx].append(0)
            else:
                score_list[idx].append(0)
    return score_list


In [28]:
def getWeightsforCommonEnglishWords():
    base_dir = 'opinion-lexicon-English'
    weights = {}
    for file in os.listdir(base_dir):
        fileid = open(base_dir+"/"+file,"r")
        for line in fileid.read().split("\n"):
            if line != "":
                words = line.split('\t')
                weights.update({words[0]:float(words[1])})
    return weights

In [29]:
#Polarity of the terms
def getWeightsMPQA():
    fileid = open("subjclueslen1-HLTEMNLP05.tff","r")
    subj_weights = {}
    for line in fileid.read().split('\n'):
        words = line.split()
        term = None
        pos_tag = None
        polarity = None
        subjectivity = None
        for word in words:
            if "word1" in word:
                term = word.split('=')[1]
            if "type" in word:
                subjectivity = word.split('=')[1]
            if "priorpolarity" in word:
                polarity = word.split('=')[1]
            if term!=None and subjectivity!=None and polarity!=None:
                if subjectivity == 'strongsubj' and polarity =='positive':
                    subj_weights.update({term:1})
                elif subjectivity == 'weaksubj' and polarity =='positive':
                    subj_weights.update({term:0.5})
                elif subjectivity == 'strongsubj' and polarity =='negative':
                    subj_weights.update({term:-1})
                elif subjectivity == 'weaksubj' and polarity =='negative':
                    subj_weights.update({term:-0.5})
                elif polarity == 'neutral':
                    subj_weights.update({term:-0.5})
    return subj_weights

In [30]:
def bigram_word_feats(words, score_fn=BigramAssocMeasures.chi_sq, n=200):
    tokenizer = WordPunctTokenizer()
    tokens = tokenizer.tokenize(words)
    
    bigram_finder = BigramCollocationFinder.from_words(tokens)
    bigrams = bigram_finder.nbest(BigramAssocMeasures.chi_sq, 500)

    for bigram_tuple in bigrams:
        x = "%s %s" % bigram_tuple
        tokens.append(x)

    result = [' '.join([w for w in x.split()]) for x in tokens if x.lower() not in stopwords.words('english') and len(x) > 8]
    return result

In [31]:
text = list(dataset[' text'])


locations = list(dataset[' term_location'])

aspect_terms = list(dataset[' aspect_term'])
#Removing comma and converting to lower
for i in range(len(text)):
    text[i] = text[i].replace('[comma]','')
    text[i] = text[i].lower()
    
reduced_sentences = []
for i in range(len(text)):
    start = int(locations[i].split("--")[0])
    end = int(locations[i].split("--")[1])
    reduced_sentences.append(getContextWords(text[i],start,end))

for i in range(len(reduced_sentences)):
    reduced_sentences[i]= ''.join(c for c in reduced_sentences[i] if c not in punctuation)
    reduced_sentences[i] = reduced_sentences[i].rstrip()



In [38]:
text = list(test[' text'])


locations = list(test[' term_location'])

aspect_terms = list(test[' aspect_term'])
#Removing comma and converting to lower
for i in range(len(text)):
    text[i] = text[i].replace('[comma]','')
    text[i] = text[i].lower()
    
reducedsentences = []
for i in range(len(text)):
    start = int(locations[i].split("--")[0])
    end = int(locations[i].split("--")[1])
    reducedsentences.append(getContextWords(text[i],start,end))

for i in range(len(reduced_sentences)):
    reducedsentences[i]= ''.join(c for c in reducedsentences[i] if c not in punctuation)
    reducedsentences[i] = reducedsentences[i].rstrip()



IndexError: list index out of range

In [32]:
print(reduced_sentences)

['any computer is the', 'good for and web browsing', 'sound when using the m', 'the same problem the ri id not work', 'my problem was with', 'fact that it is running', 'lasted 5years before the h crashed', 'to check in the s on this one before i', 'make for a very messy with cords going every direction', 'most recent being that my is freaking out on me', 'last computer a toshiba co nly  400 and worked', 'the machine has a the book has no mention', 'the is very intuitive so much', 'because of the performance to ratio', 'laptop for the price w well with action pack games', 'the is really long', 'be able to buy the', 'the battery never held a longer than one hour and', 'web access through the is so slow it s', 'used to the new s but you will learn fast', 'the is bright and the keyboard', 'the said it was under', 'me never to trust an', 'is that it has no but i found that they', 'so we went with vis', 'its a problem with this', 'was two weeks late in because hp forgot to complete', 'm tire

In [34]:
bigrams = []
for i in range(len(reduced_sentences)):
    bigrams.append(bigram_word_feats(reduced_sentences[i]))

vectorizer = TfidfVectorizer(bigrams)
vectors = vectorizer.fit_transform(reduced_sentences)
pickle.dump(vectorizer , open('countvect1.pkl','wb'))
# vectorizer = pickle.load(open('countvect1.pkl','rb'))
#counts= vectorizer.transform(reduced_sentences)

In [112]:
# #Calculating Term Weights
# weights = []
# positive,negative = loadSentimentScoresBingLiuLexicon()

# for i in range(len(reduced_sentences)):
#     weights.append({})
#     for word in reduced_sentences[i].split():
#         if word in positive:
#             weights[i].update({word:[1.0]})
#         elif word in negative:
#             weights[i].update({word:[-1.0]})
#         else:
#             weights[i].update({word:[0]})  

# score_list = getSentimentScoresSentiwordNet(reduced_sentences)
# for i in range(len(reduced_sentences)):
#     word_list = word_tokenize(reduced_sentences[i])
#     scores = score_list[i]
#     for j in range(len(word_list)):
#         curr_weight = weights[i].get(word_list[j])
#         curr_weight.append(scores[j])
#         if(curr_weight!=None):
#             weights[i].update({word_list[j] : curr_weight})
            
# review_scores = getSentimentScoresfromLexicon("Amazon-laptops-electronics-reviews-unigrams.txt")

# for i in range(len(reduced_sentences)):
#     word_list = word_tokenize(reduced_sentences[i])
#     for j in range(len(word_list)):
#         if word_list[j] in review_scores:
#             review_score = review_scores.get(word_list[j])
#             curr_weight = weights[i].get(word_list[j])
#             curr_weight.append(review_score)
#             if curr_weight!=None:
#                 weights[i].update({word_list[j] : curr_weight})

# other_weights = getWeightsforCommonEnglishWords()

# for i in range(len(reduced_sentences)):
#     word_list = word_tokenize(reduced_sentences[i])
#     for j in range(len(word_list)):
#         if word_list[j] in other_weights:
#             weight = other_weights.get(word_list[j])
#             curr_weight = weights[i].get(word_list[j])
#             curr_weight.append(weight)
#             if curr_weight!=None:
#                 weights[i].update({word_list[j] : curr_weight})

# subj_weights = getWeightsMPQA()
# for i in range(len(reduced_sentences)):
#     word_list = word_tokenize(reduced_sentences[i])
#     for j in range(len(word_list)):
#         if word_list[j] in subj_weights:
#             weight = subj_weights.get(word_list[j])
#             curr_weight = weights[i].get(word_list[j])
#             curr_weight.append(weight)
#             if curr_weight!=None:
#                 weights[i].update({word_list[j] : curr_weight})            


In [113]:
# print(weights)

[{'but': [0, 0], 'the': [0, 0, 0.027], 'was': [0, 0, -0.248], 'so': [0, 0.0, 0.1, -0.5], 'horrible': [-1.0, -0.625, -2.419, -1], 'to': [0, 0, -0.033], 'us': [0, 0, -0.098, -0.01]}, {'redeeming': [1.0, -0.041666666666666664, -2.045, 1], 'factor': [0, 0.0, 0.759, 0.54], 'was': [0, 0, 0, -0.248, -0.248], 'the': [0, 0, 0.027], 'f': [0, 0.0, 0.356, 0.57], 'which': [0, 0, 0.097], 'above': [0, 0, 0.121, 0.2, 0.5], 'average': [0, 0.0, 0.34, 0.4], 'but': [0, 0]}, {'the': [0, 0, 0.027], 'is': [0, 0.009615384615384616, 0.296], 'uniformly': [0, 0, 0.712, 0.5], 'exceptional': [1.0, 0.0, 2.343, 1], 'with': [0, 0, 0.151], 'a': [0, 0, 0.156]}, {'with': [0, 0, 0.151], 'a': [0, 0, 0.156], 'very': [0, 0.125, 0.466, -0.0, -0.5], 'capable': [1.0, 0.05, 0.857, -0.23, 0.5], 'k': [0, 0.0, -0.681, 0.57], 'which': [0, 0, 0.097], 'will': [0, 0, -0.165, 1], 'proudly': [0, 0.125, 0.019, -0.11], 'whip': [0, 0.0, 0.783, -0.23], 'up': [0, 0.075, 0.051]}, {'it': [0, 0, -0.039], 's': [0, 0, 0.023, 0.57], 'on': [0, 0, 0

In [114]:
# bigrams = []
# for i in range(len(reduced_sentences)):
#     bigrams.append(bigram_word_feats(reduced_sentences[i]))


In [39]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
x=vectors
Y=dataset[' class']
pred=train_SVC(x,Y)
preds=vectorizer.transform(reducedsentences)
new=pred.predict(preds)



Acurracy :  68.40671811166591  %

Precision Of the -1 class: 0.658696
Recall Of the -1 class: 0.731884
F1-Score Of the -1 class: 0.693364 


Precision Of the 0 class: 0.536810
Recall Of the 0 class: 0.401376
F1-Score Of the 0 class: 0.459318 


Precision Of the 1 class: 0.758621
Recall Of the 1 class: 0.773163
F1-Score Of the 1 class: 0.765823 




In [116]:
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfVectorizer
# stop_words = set(stopwords.words('english'))

# #vectorizer = TfidfVectorizer(bigrams, stop_words=stop_words)
# #counts=vectorizer.fit_transform(reduced_sentences)
# vectorizer = CountVectorizer(bigrams)
# vectors = vectorizer.fit_transform(reduced_sentences)
# pickle.dump(vectorizer , open('countvect1.pkl','wb'))
# # vectorizer = pickle.load(open('countvect1.pkl','rb'))
# counts= vectorizer.transform(reduced_sentences)
# bigram_df = pd.DataFrame(counts.toarray())

# for column in bigram_df:
#     total = bigram_df[column].sum()
#     if total < 5:
#         bigram_df = bigram_df.drop(column,axis=1)
        

In [118]:
# #Removing stopwords
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfVectorizer
# stop_words = set(stopwords.words('english'))
# vectorizer = TfidfVectorizer(bigrams)
# vectorizer.fit(reduced_sentences)
# #pickle.dump(vectorizer , open('countvect2.pkl','wb'))
# with open('countvect2.pickle', 'wb') as output:
#     pickle.dump(countvect2, output)
# #vectorizer = TfidfVectorizer(bigrams, stop_words=stop_words)
# #counts=vectorizer.fit_transform(reduced_sentences)
# #vectorizer = CountVectorizer()
# #vectorizer.fit(reduced_sentences)
# #vectorizer = pickle.load(open('countvect2.pkl','rb'))
# counts= vectorizer.transform(reduced_sentences)
# vocab = list(vectorizer.vocabulary_.keys())
# len(vocab)

NameError: name 'countvect2' is not defined

In [ ]:
# df = pd.DataFrame(0,index = range(len(reduced_sentences)),columns = vocab)

In [ ]:
# for i in range(len(df)):
#     for key,value in weights[i].items():
#         df.loc[i,key] = np.mean(value)

In [ ]:
# df=df.dropna(axis=1)
# for column in df:
#     total = df[column].sum()
#     if abs(total) < 0.5:
#         df = df.drop([column],axis=1)

In [ ]:
# df=df.dropna(axis=1)

In [ ]:
# df.describe()

In [ ]:
# from sklearn.ensemble import RandomForestClassifier, VotingClassifier
# from sklearn.svm import LinearSVC
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.naive_bayes import GaussianNB
# from sklearn.metrics import precision_recall_fscore_support
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import cross_val_predict
# from sklearn.linear_model import LogisticRegression
# from sklearn.neural_network import MLPClassifier

# import xgboost as xgb
# forest_model = RandomForestClassifier(random_state=1)
# svc = LinearSVC(random_state=0,multi_class='crammer_singer')

# nb=GaussianNB()
# lr = LogisticRegression(random_state=1)
# xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
#                 max_depth = 5, alpha = 10, n_estimators = 10)

# vc = VotingClassifier(estimators=[('rf', forest_model),('svc',svc),('lr',lr)],voting="hard")
# models = [forest_model,svc,lr,vc,nb]
# X = pd.concat([df,bigram_df],axis=1)
# Y = dataset[' class']

# preds=svc.predict(vectors)
# print(preds)
# #printScores(models,X,Y)
# #for model in models:
# #    print("Getting scores for model: "+str(model))
# #    print("==================================")
# #    preds = cross_val_predict(model,X,Y,cv=10)
# #    scores = precision_recall_fscore_support(Y,preds,labels=[-1,0,1])
# #    print("For positive class")
# #    print("Precision: "+str(scores[0][2]))
# #    print("Recall:"+str(scores[1][2]))
# #    print("Fscore:"+str(scores[2][2]))
# #    print("==================================")
# #    print("For negative class")
# #    print("Precision: "+str(scores[0][0]))
# #    print("Recall:"+str(scores[1][0]))
# #    print("Fscore:"+str(scores[2][0]))
# #    print("==================================")
# #    print("For neutral class")
# #    print("Precision: "+str(scores[0][1]))
# #    print("Recall:"+str(scores[1][1]))
# #    print("Fscore:"+str(scores[2][1]))
# #    print("==================================")
# #    print("Overall accuracy: "+str(accuracy_score(Y,preds)))
# #    print("==================================")
  

In [ ]:
#X_test=pd.concat([df,bigram_df],axis=1)

In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report
# import pickle
# #X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42)
# model1=vc.fit(X,Y)
# with open('model1.pickle', 'wb') as output:
#     pickle.dump(model1, output)

# #preds=model1.predict(X)

# scores = precision_recall_fscore_support(Y,preds,labels=[-1,0,1])
# target_names = ['negative', 'nuetral', 'positive']
# print(classification_report(Y, preds, target_names=target_names))

In [ ]:
preds

In [ ]:
dataset['example_id']

In [ ]:
with open('model1.pickle', 'rb') as model1:
    model1 = pickle.load(model1)


In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report
# import pickle
# from sklearn.metrics import accuracy_score
# preds=model1.predict(X)

# scores = precision_recall_fscore_support(Y,preds,labels=[-1,0,1])
# target_names = ['negative', 'nuetral', 'positive']
# print(classification_report(Y, preds, target_names=target_names))

In [40]:
df2=pd.DataFrame()
df2['example_id']=test['example_id']
df2['class']=new

In [ ]:
df2


In [ ]:
#np.savetxt(r'c:\Users\whisk\pred.txt', df2.values)

In [41]:
#base_filename = 'Values.txt'
with open("PredictionData1.txt","w+") as outfile:
    df2.to_string(outfile)

In [ ]:
#df2.to_csv('xgboost.txt', sep='\*\|\*', index=False, header=False)

In [43]:
myCsv = df2.astype(str).apply(lambda x: ';;'.join(x), axis=1)
myCsv.rename(';;'.join(df2.columns)).to_csv("PredictionData1.txt", header=False,index=False)

In [ ]:
#preds = cross_val_predict(model,X,Y,cv=10)
#scores = precision_recall_fscore_support(Y,preds,labels=[-1,0,1])
#target_names = ['negative', 'nuetral', 'positive']
#print(classification_report(Y, preds, target_names=target_names))
#print(scores)

In [ ]:
#params = GS.best_estimator_.get_params()
#print("best parameters found through GridSearch: ",params)

In [ ]:
X.columns

In [ ]:
Y.head()

In [ ]:
len(X.columns)

In [ ]:
len(X_test.columns)